<a href="https://colab.research.google.com/github/bok-h22/semi-project/blob/master/%EA%B3%B5%EA%B3%B5_API_%ED%99%9C%EC%9A%A9%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import json

In [ ]:
dataset= pd.read_excel('/content/법정동코드 조회자료.xlsx')
dataset

,법정동코드,구코드,동코드,법정동명,최하지역명
0,1111010100,11110,10100,서울특별시 종로구 청운동,청운동
1,1111010200,11110,10200,서울특별시 종로구 신교동,신교동
2,1111010300,11110,10300,서울특별시 종로구 궁정동,궁정동
3,1111010400,11110,10400,서울특별시 종로구 효자동,효자동
4,1111010500,11110,10500,서울특별시 종로구 창성동,창성동
...,...,...,...,...,...
462,1174010600,11740,10600,서울특별시 강동구 둔촌동,둔촌동
463,1174010700,11740,10700,서울특별시 강동구 암사동,암사동
464,1174010800,11740,10800,서울특별시 강동구 성내동,성내동
465,1174010900,11740,10900,서울특별시 강동구 천호동,천호동


In [ ]:
#법정동 자료 불러오기
아파트코드 = []

for bjd_code in range(len(dataset)):
    url = f'https://apis.data.go.kr/1613000/AptListService2/getLegaldongAptList?serviceKey=lcjsLidzgyNDrzi7pA5P0N5zo4z80gxg5NfxMgOGEQnHylY3PO5AicJgFHwU3vaR8zuiE%2B6k2O1ojfc%2FxvyMMw%3D%3D&_type=json&pageNo=1&numOfRows=1000&bjdCode={dataset[bjd_code][0]}'
    response = requests.get(url)

    data = {}

    if response.json()['response']['body']['totalCount'] == 0:
        try:            
            data['아파트명'] = '없음'
            data['아파트 code'] = '없음'
            data['법정동코드'] = dataset[bjd_code][0]
            아파트코드.append(data)
        except:
            pass

    elif response.json()['response']['body']['totalCount'] == 1:
        data['아파트명'] = response.json()['response']['body']['items']['item']['kaptName'].strip()
        data['아파트 code'] = response.json()['response']['body']['items']['item']['kaptCode']
        data['법정동코드'] = response.json()['response']['body']['items']['item']['bjdCode']
        아파트코드.append(data)
    else:
        apart_code = response.json()['response']['body']['items']['item']

        for i in apart_code:
            data = {}
            data['아파트명'] = i['kaptName']
            data['아파트 code'] = i['kaptCode']
            data['법정동코드'] = i['bjdCode']
            아파트코드.append(data)

In [ ]:
아파트코드_df = pd.DataFrame(아파트코드)
아파트코드_df.to_csv("./아파트코드.csv", encoding = 'utf-8-sig')

seoul_df = pd.merge(아파트코드_df, dataset, on = '법정동코드', how = 'left')
seoul_df.to_csv("./아파트코드_최종.csv", encoding = 'utf-8-sig')